# Проект: Исследование надежности заемщиков

___Заказчик исследования:___
кредитный отдел банка. 

___Главная цель исследования:___ 
определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

___Данные для анализа:___
статистика о платёжеспособности клиентов от банка (файл `borrower_reliability.csv`).

__Необходимо учесть:__

В зависимости от дохода банк относит клиента к одной из категорий:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

## Загрузка и обзор данных

In [1]:
import pandas as pd

In [ ]:
data = pd.read_csv('/datasets/borrower_reliability.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


__Вывод__:
- Датасет содержит 21525 записей, данные соответствуют описанию. Названия столбцов соответствуют змеиному регистру.
- Есть пропуски в столбцах `days_employed` и `total_income`. 
- В данных присутствуют аномалии - заметили отрицательное количество дней трудового стажа в столбце `days_employed`. 
- Присутствуют неявные дубликаты (например, "СРЕДНЕЕ"/"среднее").

## Предобработка данных

### Удаление пропусков

In [5]:
# сохраним количество строк исходного датасета для контроля изменения размерности
original_size = data.shape[0]

Выведем количество пропущенных значений для каждого столбца.

In [6]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах `days_employed` и `total_income` есть пропущенные значения. Пропуски могли возникнуть по техническим причинам (во время выгрузки данных, копирования, считывания, сохранения информации), из-за ошибок, допущенных при вводе, либо данные могли быть не предоставлены.

Столбец `total_income` хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`. Пропуски в столбце `days_employed` также будем заполнять медианными значениями по каждому типу занятости.

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [8]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [9]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработка аномальных значений

Отрицательное количество дней трудового стажа в столбце `days_employed` является аномалией. Заменим все отрицательные значения положительными с помощью метода `abs()`.

In [10]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа в днях.

In [11]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Оставим как есть, т.к. этот столбец не понадобится для исследования.

Посмотрим на уникальные значения столбца `children`.

In [12]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [13]:
# контролируем размер потерь при удалении аномалий
data[(data['children'] != -1) & (data['children'] != 20)].shape[0] / original_size

0.9942857142857143

Значения `-1` и `20` являются аномалией, удалим их.

In [14]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Убедимся, что артефакты удалены.

In [15]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Проверим значения возраста клиента.

In [16]:
print(sorted(data['dob_years'].unique()))

[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


In [17]:
# контролируем размер потерь при удалении аномалий
data[data['dob_years'] != 0].shape[0] / original_size

0.9896399535423925

Удалим строки с аномальными значениями.

In [18]:
data = data[data['dob_years'] != 0]

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [19]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Поработаем с неявными дубликатами. В столбце `education` есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.

In [20]:
data['education'] = data['education'].str.lower()

In [21]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [22]:
data.groupby('gender')['children'].count()

gender
F      14083
M       7218
XNA        1
Name: children, dtype: int64

Выявили аномалию `XNA`. Всего одна строка - удалим.

In [23]:
data = data[data['gender'] != 'XNA']

In [24]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [25]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Неявных дубликатов и аномалий нет, но стоит привести к единообразному написанию.

In [26]:
data['family_status'] = data['family_status'].str.lower()

Проверим наличие явных дубликатов. Удалим, если такие имеются.

In [27]:
data.duplicated().sum()

71

In [28]:
data = data.drop_duplicates()

In [29]:
# контролируем размер датасета после изменений
data.shape[0] / original_size

0.986295005807201

In [30]:
data.shape[0]

21230

### Категоризация данных

На основании диапазонов, которые используются банком, создадим столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [31]:
def categorize_income(income):
    """
    Классифицирует клиента по уровню дохода на категории от A до E.

    Параметры:
        income: Размер дохода клиента. Должен быть числовым значением.

    Возвращает:
        str: Буквенное обозначение категории дохода:
            - 'A' - доход от 1 000 001 и выше
            - 'B' - доход от 200 001 до 1 000 000
            - 'C' - доход от 50 001 до 200 000
            - 'D' - доход от 30 001 до 50 000
            - 'E' - доход от 0 до 30 000
            - None: Если входные данные некорректны (не число)
    """
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [32]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Посмотрим на перечень уникальных целей взятия кредита из столбца `purpose`.

In [33]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Некоторые цели похожи. На основании данных из столбца `purpose` сформируем новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [34]:
def categorize_purpose(row):
    """
    Классифицирует цель кредита на категории на основе ключевых слов в строке.

    Параметры:
        row (str): Строка с описанием цели кредита.

    Возвращает:
        str: Категория цели кредита:
            - 'операции с автомобилем' - если содержит 'автом'
            - 'операции с недвижимостью' - если содержит 'жил' или 'недвиж'
            - 'проведение свадьбы' - если содержит 'свад'
            - 'получение образования' - если содержит 'образов'
            - 'нет категории' - если не найдено совпадений или произошла ошибка
    """        
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [35]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

__Вывод__:
- Пропуски в столбцах `days_employed` и `total_income` заполнили медианными значениями по каждому типу занятости.
- Удалили строки, содержащие аномальные значения количества дней трудовог остажа, детей, возраста. 
- Удалили явные и неявные дубликаты.
- Выполнили категоризацию данных по уровню дохода и цели ккредитования.
- После предобработки осталось 98,6% данных - 21230 записей.

### Исследование данных

#### Зависимость между количеством детей и возвратом кредита в срок

Исследуем исходные данные и определим долю каждого из уникальных значений столбца `children`.

In [36]:
data['children'].value_counts()/data.shape[0]

children
0    0.660433
1    0.225718
2    0.096043
3    0.015450
4    0.001931
5    0.000424
Name: count, dtype: float64

С учетом полученных значений разделим данные на следующие категории:
- нет детей
- один ребенок
- двое детей
- многодетная семья (трое и более детей)

Категории запишем в новый столбец `children_category`.

In [37]:
def categorize_children(children):
    """
    Возвращает наименование категории в зависимости от количества детей children, используя правила:
    - 'нет детей', если children = 0;
    - 'один ребенок', если children = 1;
    - 'двое детей', если children = 2;
    - 'многодетная семья' — если children > 2.

    Параметры:
       children (int): Количество детей в семье.
       
    Возвращает:
        str: Категория семьи
    """    
    if children == 0:
        return 'нет детей'
    elif children == 1:
        return 'один ребенок'
    elif children == 2:
        return 'двое детей'
    elif children > 2:
        return 'многодетная семья'

In [38]:
data['children_category'] = data['children'].apply(categorize_children)

Вычислим для каждой из категорий количество клиентов, которые в нее входят, а также долю клиентов, у которых имелась задолженность по возврату кредита. В этой и последующих сводных таблицах для наглядности будем производить сортировку по долям клиентов, у которых имелась задолженность.

In [39]:
data.groupby('children_category')['debt'].agg(number_of_clients='count', share_of_debts='mean').sort_values('share_of_debts')

,number_of_clients,share_of_debts
children_category,,
нет детей,14021,0.075458
многодетная семья,378,0.082011
один ребенок,4792,0.092028
двое детей,2039,0.095145


**Вывод:**

 Клиенты, у которых нет детей, реже имеют задолженность по возврату кредита, чем клиенты, у которых есть дети. 
 
 Доля клиентов, у которых нет детей, имеющих задолженность по возврату кредита, составляет 7,5%.
 Чаще задолженность по возврату кредита возникает среди клиентов, имеющих 1 и 2 детей: 9,2% и 9,5% соответственно. 
 Вместе с тем стоит отметить, что задолженность по возврату кредита у клиентов, имеющих 3 и более детей, возникает реже, чем у клиентов, имеющих 1 или 2 детей, и составляет 8,2%. 


#### Зависимость между семейным положением и возвратом кредита в срок

Данные о семейном положении (столбец `family_status`) являются категориальными. Определим для каждой категорий количество клиентов, которые в нее входят, и долю клиентов, у которых имелась задолженность по возврату кредита.

In [40]:
data.groupby('family_status')['debt'].agg(number_of_clients='count', share_of_debts='mean').sort_values('share_of_debts')

,number_of_clients,share_of_debts
family_status,,
вдовец / вдова,946,0.065539
в разводе,1179,0.071247
женат / замужем,12213,0.075575
гражданский брак,4112,0.093142
не женат / не замужем,2780,0.097842


**Вывод:** 

Незамужние/неженатые клиенты, а также клиенты, состоящие в гражданском браке, чаще имеют задолженность по возврату кредита. 
Это наблюдается в 9,8% и 9,3% случаев соответственно.

Реже имеют задолженность по возврату кредита клиенты, состоящие в официальном браке - 7,6%, они состоявляют самую многочисленную категорию. Клиенты в разводе имеют задолженность в 7,1% случаев.

Наименьший процент должников 6,6% - среди вдов/вдовцов.


#### Зависимость между уровнем дохода и возвратом кредита в срок

Для анализа зависимости будем использовать данные из созданного выше столбца `total_income_category`, в котором клиенту определялась категория в зависимости от дохода:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

Определим для каждой категории количество клиентов, которые в нее входят, и долю клиентов, у которых имелась задолженность по возврату кредита.

In [41]:
data.groupby('total_income_category')['debt'].agg(number_of_clients='count', share_of_debts='mean').sort_values('share_of_debts')

,number_of_clients,share_of_debts
total_income_category,,
D,347,0.060519
B,4987,0.070784
A,25,0.080000
C,15849,0.084926
E,22,0.090909


**Вывод:** 

Количество клиентов данной выборки, которые были отнесены к категориям 'A' и 'E', не позволяет сделать какие-либо выводы о наличии зависимости между уровнем дохода и возвратом кредита в срок для данных категорий.

Касательно остальных категорий можно сделать вывод, что клиенты самой многочисленной категории 'С' чаще других имеют задолженность по возврату кредита - 8,5%, наименьший процент должников среди клиентов категории 'D' - 6%. Для клиентов категории 'B' этот показатель составляет 7,1%.


#### Влияние цели кредита на его возврат в срок

Для данного исследования будем группировать данные по столбцу `purpose_category`.

In [42]:
# для создания сводной таблицы используем метод pivot_table
data_pivot = data.pivot_table(index='purpose_category',  values='debt', aggfunc=['count', 'mean']).sort_values(by=[('mean','debt')])
# переименуем столбцы для лучшего восприятия итоговой таблицы
data_pivot.columns = ['number_of_clients','share_of_debts']
data_pivot

,number_of_clients,share_of_debts
purpose_category,,
операции с недвижимостью,10703,0.072596
проведение свадьбы,2299,0.078730
получение образования,3970,0.092947
операции с автомобилем,4258,0.093236


**Вывод:** 

Наиболее часто проблемы с возвратом кредита в срок возникают у клиентов, которые берут кредит с целью получения образования или проведения операций с автомобилем - в 9,3% случаев. Наименьший процент должников среди тех, кто берет кредит для проведения операций с недвижимостью - 7,2%. Задолженность возврата по кредиту в срок у клиентов, целью кредитования которых является проведение свадьбы - 7,9%.

### Общий вывод

Для проведения данного исследования использовалась статистика о платёжеспособности клиентов банка. 

Во время предобработки данных пропуски об общем трудовом стаже в днях и ежемесячном доходе были заполнены медианным значением в зависимости от типа занятости. Обработаны аномальные значения, которые присутствовали в информации об общем трудовом стаже в днях (отрицательные значения заменены на положительные). Строки, содержащие аномалии в данных о количестве детей, удалены из выборки. Данные о ежемесячном доходе привели к целочисленному типу. Данные об уровне образования привели к нижнему регистру, после чего удалили дубликаты. Произведена категоризация данных по ежемесячному доходу, цели получения кредита, количеству детей.

Стоит отметить, что выборки на основе категорий по ежемесячному доходу, предложенные заказчиком, крайне несбалансированны.

Подтвердилось наличие зависимости между количеством детей и возвратом кредита в срок: 
* клиенты, у которых нет детей, реже имеют задолженность по возврату кредита, чем клиенты, у которых есть дети,
* задолженность по возврату кредита чаще возникает у клиентов, имеющих 1 или 2 детей, чем у клиентов, имеющих 3 и более детей.

 Подтвердилось наличие зависимости между семейным положением и возвратом кредита в срок:
 * незамужние/неженатые клиенты, а также клиенты, состоящие в гражданском браке, чаще имеют задолженность по возврату кредита, чем состоящие в официальном браке либо в разводе,
  * наименьший процент должников - среди вдов/вдовцов.

Подтвердилось наличие зависимости между уровнем дохода и возвратом кредита в срок:
* клиенты с ежемесячным уровнем дохода 50001–200000 чаще имеют задолженность по возврату кредита, чем клиенты с ежемесячным уровнем дохода 200001–1000000,
* наименьший процент должников - среди клиентов с ежемесячным уровнем дохода 30001–50000.

Цели кредита влияют на его возврат в срок следующим образом:
* чаще всего задолженность возникает по кредитам с целью получения образования или проведения операций с автомобилем,
* наименьшее число должников среди тех, кто берет кредит для проведения операций с недвижимостью.

Пропуски в исходных данных могли возникнуть по техническим причинам (во время выгрузки данных, копирования, считывания, сохранения информации), из-за ошибок, допущенных при вводе, либо данные могли быть не предоставлены. Заполнение пропусков для количественных переменных медианным значением дает более объективную информацию в случае, когда в выборке есть значения, отражающие действительность, которые сильно выделяются среди всех остальных, а также при наличии аномалий в данных.